In [15]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


In [7]:
@tool
def web_loader(url: str) -> str:
    """抓取url对应网页的内容"""
    loader = WebBaseLoader(url)
    docs = loader.load()
    return docs[0].page_content

In [8]:
llm = ChatOpenAI(model="gpt-3.5-turbo", max_tokens=4096)
tools = [web_loader]
llm_with_tools = llm.bind_tools(tools)


In [28]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你是一个擅长对长文做总结的智能助手，可以精确提炼出长文中的要点。注意，请使用markdown格式返回结果。",
        ),
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [29]:
prompt

ChatPromptTemplate(input_variables=['input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': [], 'agent_scratchpad': []}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='你是一个擅长对长文做总结的智能助手，可以精确提炼出长文中的要点。注意，请使用markdown格式返回结果。')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate

In [18]:
agent = create_tool_calling_agent(llm_with_tools, tools, prompt)

In [19]:

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

In [20]:
res = agent_executor.invoke(
    {
        "input": "这个链接讲了什么内容？ https://blog.csdn.net/xindoo/article/details/138356308?spm=1001.2014.3001.5501"
    }
)

In [30]:
res

{'input': '这个链接讲了什么内容？ https://blog.csdn.net/xindoo/article/details/138356308?spm=1001.2014.3001.5501',
 'output': '这篇博文介绍了一个命令行工具ShellGPT，它是一个开源软件，可以帮助用户快速生成 shell 命令、代码片段和文档，无需再依赖外部资源（如 Google 搜索）。以下是该文章的主要内容：\n\n- ShellGPT的配置安装：支持 Linux、macOS、Windows 等操作系统，安装简单，只需使用pip安装即可。\n- 函数调用：ShellGPT允许执行系统中的函数，如播放音乐、打开网页或执行 shell 命令。提供了一些高级配置选项。\n- 角色管理：用户可以创建自定义角色，这些角色可以用来生成代码、执行 shell 命令或满足特定需求。用户可以使用命令指定特定的角色执行指令。\n- 聊天功能：用户可以和GPT进行聊天，支持单轮聊天和多轮聊天模式。\n- 写命令并执行：用户可以借助GPT的编码能力写脚本或命令，并执行。\n- 其他功能：文章提及了一些其他细节和功能，如如何通过 API从 0 到 1 构建本地 GPTs、GPT每日面试题等。\n\n总的来说，ShellGPT是一个实用的工具，可以帮助用户在命令行环境下更高效地生成命令和代码，同时还提供了一些高级功能和配置选项。'}